In [5]:
### Defining functions to scrape website & food blogs
## 1. With application/ld+json
## 2. With schema tags itemprop
## 3. With unstructured format 




In [186]:
json_ld: [
    "https://www.marmiton.org/recettes/recette_gratin-dauphinois-tres-facile_58956.aspx",
    "https://clemfoodie.com/2021/08/22/crumble-aux-prunes-rouges-et-amandes/",
    "https://tangerinezest.com/pizza-tomate-burrata-basilic-et-mortadelle/",
    "https://recettesdejulie.fr/10351/crumble-mirabelles-avoine-noisettes/"
]
    
microdata: [
    "https://mesbrouillonsdecuisine.fr/pancakes-sales-a-la-farine-de-pois-chiches-jeunes-pousses-depinards-et-tomates-sechees/"
]
    
hard_test_urls: [
    "https://www.undejeunerdesoleil.com/2018/05/tramezzini-thon-artichauts-venise.html",
    "http://www.chezmisa.com/burgers-de-boeuf-persilles-et-sauce-au-miel/",
    "https://cookingjulia.blogspot.com/2021/08/poulet-la-mexicaine.html",
    "https://wernerhappyeats.com/djeunerdner/2017/12/29/grilled-chicken-lunch-bento-yxbn2",
    "https://doriannn.blogspot.com/2021/08/knackinkorea-parce-que-decidement-je-ne.html",
    "https://madamcadamia.com/2021/09/02/crumble-aux-mures-et-noisettes/",
    "https://www.plusunemiettedanslassiette.fr/moules-marinara/"
]

In [163]:
import pandas as pd
import numpy as np
import requests
import extruct
import pprint
from w3lib.html import get_base_url

In [440]:
url = 'https://yuka.io/recettes/banana-bread/'
recipe = scrape(url)
print(recipe)
#print(recipe['recipe']['yield'].item())

{'recipe': {'name': 0    Banana bread au beurre de cacahuètes
Name: name, dtype: object, 'yield': 0    1 cake 
Name: recipeYield, dtype: object, 'ingredients': 0    [3 bananes bien mûres , 3 œufs , 100g de beurr...
Name: recipeIngredient, dtype: object}}


# 1. Scrapping JSON-LD OR MICRODATA

In [393]:
"""Fetch structured JSON-LD OR MICRODATA data from a given URL."""
from typing import Optional, List
import requests
import extruct
from w3lib.html import get_base_url
from bs4 import BeautifulSoup


def scrape(url: str) -> Optional[List[dict]]:
    """Parse structured data from a URL."""
    headers = {
        'Access-Control-Allow-Origin': '*',
        'Access-Control-Allow-Methods': 'GET',
        'Access-Control-Allow-Headers': 'Content-Type',
        'Access-Control-Max-Age': '3600',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }    
    req = requests.get(url, headers=headers)
    base_url = get_base_url(req.content, url)
    html = get_html(url) 
    metadata = get_metadata(html, url)
    if metadata:
        recipe_df = get_recipe_df(metadata)
        recipe = {"recipe": {
                     "name": recipe_df.name,
                     "yield": recipe_df.recipeYield,
                     "ingredients": recipe_df.recipeIngredient    
                    }
                }            
    else:
        recipe = {"no results"
                }        
    return recipe


def get_html(url: str):
    """Get raw HTML from a URL."""
    headers = {
        'Access-Control-Allow-Origin': '*',
        'Access-Control-Allow-Methods': 'GET',
        'Access-Control-Allow-Headers': 'Content-Type',
        'Access-Control-Max-Age': '3600',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }
    req = requests.get(url, headers=headers)
    return req.text


def get_metadata(html, url: str):
    r = requests.get(url)
    """Fetch JSON-LD structured data."""
    metadata = extruct.extract(
        html,
        base_url = get_base_url(r.text, r.url),
        syntaxes=['json-ld'],
        uniform=True
    )['json-ld']
    """If empty, try fetch Microdata structured data."""    
    if metadata == []:
        metadata = extruct.extract(
            html,
            base_url = get_base_url(r.text, r.url),
            syntaxes=['microdata'],
            uniform=True
        )['microdata']        
    if bool(metadata) and isinstance(metadata, list):
        metadata = metadata[0]
    return metadata

def get_recipe_df(metadata):
    #check metadata dict format 
    if '@graph' in metadata:
        recipe_df = pd.DataFrame(metadata['@graph'])
    else:
        recipe_df = pd.DataFrame.from_dict(metadata, orient='index').transpose()
    #get first row with recipeIngredient
    recipe_df = recipe_df.sort_values(by='recipeIngredient').head(1)
    return recipe_df


# 2. Non-structured data

In [20]:
import scrape_schema_recipe

In [360]:
url = 'https://www.byacb4you.com/number-cake-noisette-ganache-pralinoise.html'
recipe_list = scrape_schema_recipe.scrape_url(url, python_objects=True)
recipe = recipe_list[0]

print(recipe['name'])
print(recipe['recipeIngredient'])


Number cake, biscuit noisette et ganache Pralinoise
['4  œufs', '160 g de poudre de noisette', '160 g de sucre glace', '40 g de beurre fondu', '4  blancs d’œuf', '25 g de sucre', '60 g de farine', '50 cl de crème fleurette', '360 g de Pralinoise (2 tablettes)', '40 g de chocolat noir', '100 g de mûres', 'Amandes enrobées de chocolat', 'Crêpes dentelle au chocolat', 'Quelques fleurs de Souci']
['20', '20 parts']


In [411]:
hard_test_urls = [
    "https://www.undejeunerdesoleil.com/2018/05/tramezzini-thon-artichauts-venise.html",
    "http://www.chezmisa.com/burgers-de-boeuf-persilles-et-sauce-au-miel/",
    "https://cookingjulia.blogspot.com/2021/08/poulet-la-mexicaine.html",
    "https://wernerhappyeats.com/djeunerdner/2017/12/29/grilled-chicken-lunch-bento-yxbn2",
    "https://doriannn.blogspot.com/2021/08/knackinkorea-parce-que-decidement-je-ne.html",
    "https://madamcadamia.com/2021/09/02/crumble-aux-mures-et-noisettes/",
    "https://www.plusunemiettedanslassiette.fr/moules-marinara/",
    "https://www.instagram.com/p/CTU03RgCaZv/"
]

In [581]:
pp = pprint.PrettyPrinter(indent=2)
r = requests.get("https://clemfoodie.com/2021/08/22/crumble-aux-prunes-rouges-et-amandes/")
base_url = get_base_url(r.text, r.url)
data = extruct.extract(r.text, base_url=base_url)

pp.pprint(data)

{ 'dublincore': [ { 'elements': [ { 'URI': 'http://purl.org/dc/elements/1.1/description',
                                    'content': "Une recette de crumble d'été "
                                               'aux prunes rouges avec une '
                                               'pâte à crumble aux amandes ! '
                                               "C'est facile à réaliser et "
                                               'délicieux !',
                                    'name': 'description'}],
                    'namespaces': {},
                    'terms': []}],
  'json-ld': [ { '@context': 'https://schema.org',
                 '@graph': [ { '@id': 'https://clemfoodie.com/#website',
                               '@type': 'WebSite',
                               'description': '',
                               'inLanguage': 'fr-FR',
                               'name': 'Clemfoodie',
                               'potentialAction': [ { '@type': 'Search

In [414]:
from trafilatura import fetch_url, extract
import trafilatura 

In [696]:
url = "https://www.plusunemiettedanslassiette.fr/moules-marinara/"
downloaded = fetch_url(url)

In [697]:
# to get the page title ?
if downloaded is not None:
    metadata = trafilatura.metadata.extract_metadata(downloaded)
    #metadata = trafilatura.core.baseline(downloaded)
    print(metadata)

{'title': 'Moules marinara', 'author': 'LadyMilonguera dit', 'url': 'https://www.plusunemiettedanslassiette.fr/moules-marinara/', 'hostname': 'plusunemiettedanslassiette.fr', 'description': None, 'sitename': 'plusunemiettedanslassiette.fr', 'date': '2021-08-26', 'categories': [], 'tags': [], 'fingerprint': None, 'id': None, 'license': None}


In [698]:
# to eg the main text of a page
if downloaded is not None:
    result = extract(downloaded, include_comments=False, target_language="fr")
    print(result)

Je ne sais pas vous, mais s’il y a bien une chose que j’adore manger l’été, ce sont bien les moules ! Elles m’évoquent les vacances, la mer, le soleil et mes souvenirs d’enfance à la plage. Manger des moules l’été, c’est un peu comme un rituel pour moi car, oui, je fais partie de cette catégorie de personnes qui ne peuvent pas rentrer de vacances sans avoir mangé une bonne moule – frite. Et je suis sûre que je ne suis pas la seule dans ce cas…
Bref, vous l’aurez compris, j’ai une vraie passion pour ce petit coquillage. Donc quand Grand Frais m’a proposé d’imaginer une recette à base de moules, j’étais plus que ravie. Je n’en avais encore jamais posté par ici, allez savoir pourquoi… J’ai donc filé chez Grand Frais cette semaine pour acheter de bonnes moules de bouchot Label Rouge, les meilleures selon moi, elles sont de nature petites mais très savoureuses, et je me suis lancée dans la préparation d’une recette aux saveurs italiennes. Bah oui, je me doute que la recette de la marinière 

In [442]:
from nltk.tokenize import sent_tokenize


In [443]:
text = result

In [466]:
sentences = sent_tokenize(text, language='french')

In [473]:
new_lines_split_sentences = [sentence.split('\n') for sentence in sentences]
clean_sentences = [item for sublist in new_lines_split_sentences for item in sublist]


'De plus ce sont des fleurs comestibles qui même si elles n’ont pas été mangées, ont terminé dans ma tisane.'

In [474]:
len(clean_sentences)

92

In [475]:
clean_sentences[70]

'- Mettez 10 cl de crème à bouillir.'

In [723]:
# export file to disk
label_data = pd.read_csv(r'/Users/vincentsalamand/Documents/datasets/label_recipe_text.csv')
label_data.head()

,text,type
0,Huile de friture,ingredient
1,Sel (Poivre),ingredient
2,poivre,ingredient
3,1 cuillère à soupe de vinaigre de riz,ingredient
4,Herbes de Provence,ingredient


In [724]:
label_data[label_data.isnull().any(axis=1)].tail(20)


,text,type
82553,NaN,instruction
82555,Presser les 2/3 de la pâte uniformément au fon...,NaN
82556,NaN,instruction
82557,Réserver le 1/3 restant.,NaN
82558,NaN,instruction
82561,Enfourner 35 minutes jusqu'à ce que la pâte so...,NaN
82562,NaN,instruction
82563,Laisser refroidir complètement et réfrigérer.,NaN
82564,NaN,instruction
83991,NaN,instruction
